#  This file finds the region of interest (ROI) size for the lesions 
## It is organised as follows
 * fit convex hull to each segmentation mask
 * fit smallest bounding box around convex hulls
 * caluclate maximum size of bounding box in dataset 
 * find how many are above 64x64


In [2]:
import argparse
import os
import numpy as np

from medpy.filter.smoothing import anisotropic_diffusion
from scipy.ndimage import median_filter
from skimage import measure, morphology
import scipy.ndimage as ndimage
from sklearn.cluster import KMeans

import sys
import os
from pathlib import Path
import glob
from configparser import ConfigParser
import pandas as pd
import warnings
import pylidc as pl
from tqdm import tqdm
from statistics import median_high

#from utils import is_dir_path,segment_lung
from pylidc.utils import consensus
from PIL import Image


from scipy import ndimage as ndi
from scipy.ndimage import label, generate_binary_structure
from sklearn.metrics.pairwise import euclidean_distances

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns

# Maths
import math
from scipy.spatial import ConvexHull
from math import sqrt
import numpy as np
from math import atan2, cos, sin, pi
from collections import namedtuple

In [3]:
IMAGE_DIR = r"C:\Users\mm17b2k\Documents\ARCANE\Python\MSc\Data_full\Images"
MASK_DIR =  r"C:\Users\mm17b2k\Documents\ARCANE\Python\MSc\Data_full\Mask"
CLEAN_DIR_IMG = r"C:\Users\mm17b2k\Documents\ARCANE\Python\MSc\Data_full\Clean\Images"
CLEAN_DIR_MASK = r"C:\Users\mm17b2k\Documents\ARCANE\Python\MSc\Data_full\Clean\Mask"

In [4]:
#important functions: MinimumBoundingBox

def unit_vector(pt0, pt1):
    ''' returns an unit vector that points in the direction of pt0 to pt1 '''
    dis_0_to_1 = sqrt((pt0[0] - pt1[0])**2 + (pt0[1] - pt1[1])**2)
    return (pt1[0] - pt0[0]) / dis_0_to_1, \
           (pt1[1] - pt0[1]) / dis_0_to_1


def orthogonal_vector(vector):
    ''' from vector returns a orthogonal/perpendicular vector of equal length '''
    return -1 * vector[1], vector[0]


def bounding_area(index, hull):
    '''calculate statistics from bounding box'''
    unit_vector_p = unit_vector(hull[index], hull[index+1])
    unit_vector_o = orthogonal_vector(unit_vector_p)

    dis_p = tuple(np.dot(unit_vector_p, pt) for pt in hull)
    dis_o = tuple(np.dot(unit_vector_o, pt) for pt in hull)

    min_p = min(dis_p)
    min_o = min(dis_o)
    len_p = max(dis_p) - min_p
    len_o = max(dis_o) - min_o

    return {'area': len_p * len_o,
            'length_parallel': len_p,
            'length_orthogonal': len_o,
            'rectangle_center': (min_p + len_p / 2, min_o + len_o / 2),
            'unit_vector': unit_vector_p,
            }


def to_xy_coordinates(unit_vector_angle, point):
    '''returns converted unit vector coordinates in x, y coordinates'''
    angle_orthogonal = unit_vector_angle + pi / 2
    return point[0] * cos(unit_vector_angle) + point[1] * cos(angle_orthogonal), \
           point[0] * sin(unit_vector_angle) + point[1] * sin(angle_orthogonal)


def rotate_points(center_of_rotation, angle, points):
    ''' 
    This requires:
    - center_of_rotation to be a 2d vector. e.g.: (1.56, -23.4)
    - angle to be in radians
    - points to be a list or tuple of points. ex: ((1.56, -23.4), (1.56, -23.4))
    
    This function returns: 
    a point cloud rotated around the center_of_rotation point by an angle
    '''
    rot_points = []
    ang = []
    for pt in points:
        diff = tuple([pt[d] - center_of_rotation[d] for d in range(2)])
        diff_angle = atan2(diff[1], diff[0]) + angle
        ang.append(diff_angle)
        diff_length = sqrt(sum([d**2 for d in diff]))
        rot_points.append((center_of_rotation[0] + diff_length * cos(diff_angle),
                           center_of_rotation[1] + diff_length * sin(diff_angle)))

    return rot_points


def rectangle_corners(rectangle):
    '''
    Input: the output of mon_bounding_rectangle
    Reurns:the corner locations of the bounding rectangle
    '''
    corner_points = []
    for i1 in (.5, -.5):
        for i2 in (i1, -1 * i1):
            corner_points.append((rectangle['rectangle_center'][0] + i1 * rectangle['length_parallel'],
                            rectangle['rectangle_center'][1] + i2 * rectangle['length_orthogonal']))

    return rotate_points(rectangle['rectangle_center'], rectangle['unit_vector_angle'], corner_points)


BoundingBox = namedtuple('BoundingBox', ('area',
                                         'length_parallel',
                                         'length_orthogonal',
                                         'rectangle_center',
                                         'unit_vector',
                                         'unit_vector_angle',
                                         'corner_points'
                                        )
)


# use this function to find the listed properties of the minimum bounding box of a point cloud
def MinimumBoundingBox(points):
    '''
    Requires: points to be a list or tuple of 2D points. e.g.: ((5, 2), (3, 4), (6, 8)), needs to be more than 2 points
    
    Returns a tuple that contains:
     - area: area of the rectangle
     - length_parallel: length of the side that is parallel to unit_vector
     - length_orthogonal: length of the side that is orthogonal to unit_vector
     - rectangle_center: coordinates of the rectangle center
       (use rectangle_corners to get the corner points of the rectangle)
     - unit_vector: direction of the length_parallel side. RADIANS
       (it's orthogonal vector can be found with the orthogonal_vector function
     - unit_vector_angle: angle of the unit vector
     - corner_points: set that contains the corners of the rectangle
    '''
    if len(points) <= 2: raise ValueError('More than two points required.')

    hull_ordered = [points[index] for index in ConvexHull(points).vertices]
    hull_ordered.append(hull_ordered[0])
    hull_ordered = tuple(hull_ordered)

    min_rectangle = bounding_area(0, hull_ordered)
    for i in range(1, len(hull_ordered)-1):
        rectangle = bounding_area(i, hull_ordered)
        if rectangle['area'] < min_rectangle['area']:
            min_rectangle = rectangle

    min_rectangle['unit_vector_angle'] = atan2(min_rectangle['unit_vector'][1], min_rectangle['unit_vector'][0])
    min_rectangle['rectangle_center'] = to_xy_coordinates(min_rectangle['unit_vector_angle'], min_rectangle['rectangle_center'])

    return BoundingBox(
        area = min_rectangle['area'],
        length_parallel = min_rectangle['length_parallel'],
        length_orthogonal = min_rectangle['length_orthogonal'],
        rectangle_center = min_rectangle['rectangle_center'],
        unit_vector = min_rectangle['unit_vector'],
        unit_vector_angle = min_rectangle['unit_vector_angle'],
        corner_points = set(rectangle_corners(min_rectangle))
    )

def is_dir_path(string):
    if os.path.isdir(string):
        return string
    else:
        raise NotADirectoryError(string)

In [5]:
def find_bbox_len(mask_img_slice):
    '''find the size of the bounding box by x and y length'''
   # mask_img_slice = img_slice.replace('NI','MA')

    mask_img = np.load(MASK_DIR+mask_img_slice)
    mask_index_matrix = np.where(mask_img == True)
    coords = []
    for i in range(len(mask_index_matrix[0])):
        # add all coords of mask to list [x,y]
        coords.append([mask_index_matrix[0][i],mask_index_matrix[1][i]])
        
        
    bbox = np.array([[x,y] for x,y in list(MinimumBoundingBox(coords)[6])])
    
    # find straight orientated bounding box coordinates
    x_min = min(point[0] for point in bbox)
    x_max = max(point[0] for point in bbox)
    y_min = min(point[1] for point in bbox)
    y_max = max(point[1] for point in bbox)
    
    x_len = x_max - x_min
    
    y_len = y_max - y_min
    
    return x_len, y_len

In [6]:
DICOM_DIR = r"C:\Users\mm17b2k\Documents\ARCANE\Python\MSc\Data_full\Mask"
LIDC_IDRI_list= [f for f in os.listdir(DICOM_DIR) if not f.startswith('.')]
LIDC_IDRI_list.sort()
if 'LICENSE' in LIDC_IDRI_list:
    LIDC_IDRI_list.remove('LICENSE')

all_files_list = []
for i in range(len(LIDC_IDRI_list)):
    patient_slice_list = sorted([f for f in os.listdir(DICOM_DIR + '/' + LIDC_IDRI_list[i]) if not f.startswith('.')])
    for j in range(len(patient_slice_list)):
        all_files_list.append('/LIDC-IDRI-'+ patient_slice_list[j][0:4] + '/' + patient_slice_list[j])
        
print(len(all_files_list))
print(all_files_list[0:5])

13916
['/LIDC-IDRI-0001/0001_MA000_slice000.npy', '/LIDC-IDRI-0001/0001_MA000_slice001.npy', '/LIDC-IDRI-0001/0001_MA000_slice002.npy', '/LIDC-IDRI-0001/0001_MA000_slice003.npy', '/LIDC-IDRI-0001/0001_MA000_slice004.npy']


In [7]:
all_files_list[-1]

'/LIDC-IDRI-1012/1012_MA000_slice002.npy'

In [8]:
all_files_list.remove(all_files_list[9846])

In [9]:
def find_max_bbox(file_list):
    x_list, y_list, i_list = [], [], []
    for i, file in enumerate(file_list):
        x, y = find_bbox_len(file)
        x_list.append(x)
        y_list.append(y)
        i_list.append(i)

    print(max(x_list), max(y_list))
    return math.ceil(max(x_list)), math.ceil(max(y_list)), x_list, y_list, i_list

max_bbox = find_max_bbox(all_files_list)

71.52702702702703 71.95081967213122


In [10]:
max_bbox[0:2]

(72, 72)

In [11]:
bigger = []
x_list, y_list = max_bbox[2:4]
for i, x in enumerate(x_list):
    if x > 64:
        print('x',i, x)
        bigger.append([i,x])
        
for i, y in enumerate(y_list):
    if y > 64:
        print('y',i, y)   
        bigger.append([i,y])


x 4035 67.00000000000011
x 4037 67.0
x 5401 67.97160883280759
x 5402 71.52702702702703
x 7293 69.0
x 7322 64.60000000000002
x 7323 64.15094339622647
x 8715 64.19999999999993
x 8716 64.42021924482333
x 8717 68.1817034700315
x 8718 64.26875369167158
y 4038 65.30254184748924
y 5401 67.75394321766565
y 5402 70.68918918918921
y 5403 65.19999999999997
y 7293 69.0
y 9377 67.48047722342739
y 9378 67.97093791281372
y 9379 65.90449438202239
y 9380 69.93013698630136
y 9381 71.95081967213122
y 9382 70.66153846153844
y 9383 65.48648648648646
y 9384 66.89344262295083


In [12]:
print(len(bigger))
i_list = max_bbox[4]
list2 = []
i_list2 = []
for a in bigger:
    list2.append(all_files_list[i_list[a[0]]])
    i_list2.append(a[0])
list2

24


['/LIDC-IDRI-0337/0337_MA001_slice006.npy',
 '/LIDC-IDRI-0337/0337_MA001_slice008.npy',
 '/LIDC-IDRI-0436/0436_MA000_slice006.npy',
 '/LIDC-IDRI-0436/0436_MA000_slice007.npy',
 '/LIDC-IDRI-0575/0575_MA000_slice000.npy',
 '/LIDC-IDRI-0576/0576_MA000_slice019.npy',
 '/LIDC-IDRI-0576/0576_MA000_slice020.npy',
 '/LIDC-IDRI-0661/0661_MA004_slice015.npy',
 '/LIDC-IDRI-0661/0661_MA004_slice016.npy',
 '/LIDC-IDRI-0661/0661_MA004_slice017.npy',
 '/LIDC-IDRI-0661/0661_MA004_slice018.npy',
 '/LIDC-IDRI-0337/0337_MA001_slice009.npy',
 '/LIDC-IDRI-0436/0436_MA000_slice006.npy',
 '/LIDC-IDRI-0436/0436_MA000_slice007.npy',
 '/LIDC-IDRI-0436/0436_MA000_slice008.npy',
 '/LIDC-IDRI-0575/0575_MA000_slice000.npy',
 '/LIDC-IDRI-0703/0703_MA001_slice012.npy',
 '/LIDC-IDRI-0703/0703_MA001_slice013.npy',
 '/LIDC-IDRI-0703/0703_MA001_slice014.npy',
 '/LIDC-IDRI-0703/0703_MA001_slice015.npy',
 '/LIDC-IDRI-0703/0703_MA001_slice016.npy',
 '/LIDC-IDRI-0703/0703_MA001_slice017.npy',
 '/LIDC-IDRI-0703/0703_MA001_sli

In [13]:
files = []
for a in bigger:
    files.append(all_files_list[a[0]][16:])
files

['0337_MA001_slice006.npy',
 '0337_MA001_slice008.npy',
 '0436_MA000_slice006.npy',
 '0436_MA000_slice007.npy',
 '0575_MA000_slice000.npy',
 '0576_MA000_slice019.npy',
 '0576_MA000_slice020.npy',
 '0661_MA004_slice015.npy',
 '0661_MA004_slice016.npy',
 '0661_MA004_slice017.npy',
 '0661_MA004_slice018.npy',
 '0337_MA001_slice009.npy',
 '0436_MA000_slice006.npy',
 '0436_MA000_slice007.npy',
 '0436_MA000_slice008.npy',
 '0575_MA000_slice000.npy',
 '0703_MA001_slice012.npy',
 '0703_MA001_slice013.npy',
 '0703_MA001_slice014.npy',
 '0703_MA001_slice015.npy',
 '0703_MA001_slice016.npy',
 '0703_MA001_slice017.npy',
 '0703_MA001_slice018.npy',
 '0703_MA001_slice019.npy']

In [14]:
file_list = [4035, 4037, 4038, 5401, 5402, 5403, 6200, 6201, 6202, 6203, 6204, 6205, 6206, 6207, 6208, 6209, 6210, 6211, 6212, 7293, 7322, 7323, 8715, 8716, 8717, 8718, 9377, 9378, 9379, 9380, 9381, 9382, 9383, 9384, 13040, 13041, 13042, 13043, 13055, 13056]
files = []
for a in file_list:
    b = all_files_list[a][16:]
    b = b.replace('MA', 'NI')
    files.append(b)
print(files)

['0337_NI001_slice006.npy', '0337_NI001_slice008.npy', '0337_NI001_slice009.npy', '0436_NI000_slice006.npy', '0436_NI000_slice007.npy', '0436_NI000_slice008.npy', '0487_NI000_slice002.npy', '0487_NI000_slice003.npy', '0487_NI000_slice004.npy', '0487_NI000_slice005.npy', '0487_NI000_slice006.npy', '0487_NI000_slice007.npy', '0487_NI000_slice008.npy', '0487_NI000_slice009.npy', '0487_NI000_slice010.npy', '0487_NI000_slice011.npy', '0487_NI000_slice012.npy', '0487_NI000_slice013.npy', '0487_NI000_slice014.npy', '0575_NI000_slice000.npy', '0576_NI000_slice019.npy', '0576_NI000_slice020.npy', '0661_NI004_slice015.npy', '0661_NI004_slice016.npy', '0661_NI004_slice017.npy', '0661_NI004_slice018.npy', '0703_NI001_slice012.npy', '0703_NI001_slice013.npy', '0703_NI001_slice014.npy', '0703_NI001_slice015.npy', '0703_NI001_slice016.npy', '0703_NI001_slice017.npy', '0703_NI001_slice018.npy', '0703_NI001_slice019.npy', '0951_NI000_slice001.npy', '0951_NI000_slice002.npy', '0951_NI000_slice003.npy', 

In [1]:
file_list = [4035, 4037, 4038, 5401, 5402, 5403, 6200, 6201, 6202, 6203, 6204, 6205, 6206, 6207, 6208, 6209, 6210, 6211, 6212, 7293, 7322, 7323, 8715, 8716, 8717, 8718, 9377, 9378, 9379, 9380, 9381, 9382, 9383, 9384, 13040, 13041, 13042, 13043, 13055, 13056]
len(file_list)

40